# 인공지능 과제4
### : Naive Bayes 기반 문장 감성분류
- 담당 교수님: 김학수 교수님
- 학번: 201711719
- 학과: 응용통계학과
- 이름: 심은선
- 제출날짜: 2020.09.27

# 1. 필요 함수 정의

### a) TF-IDF vocab 추출 함수 정의

In [1]:
from typing import List
import re
import collections
import numpy as np

In [2]:
# 문서별 TF-IDF 구하기
def get_TFIDF(docs: List):

    # 문서별 TF 구하기 (문서별로 단어 등장 횟수 count, 공백 기준 분리)
    def get_TF(docs: List):
        TF = [] 
        for idx, text in enumerate(docs):
            words = text.split()
            word_count = collections.Counter(words)
            TF.append(word_count)
        return TF

    # DF 구하기
    def get_DF(docs: List):
        word_ls = []
        for doc in docs:
            word_ls += list(set(doc.split()))
        df = collections.Counter(word_ls) #문서 전체의 df
        return df
    
    tf = get_TF(docs)
    df = get_DF(docs)
    N = len(docs)
    
    docs_tf_idf = []
    for word_count in tf: #문서마다 반복
        doc_tf_idf = dict()
        for word, count in word_count.items(): # 문서의 단어별로 tf-idf를 구한다
            word_tf_idf = count * np.log(N / df[word])
            doc_tf_idf[word] = word_tf_idf
        docs_tf_idf.append(doc_tf_idf)
    return docs_tf_idf


# tf_idf가 높은 상위 k개의 단어 리스트 얻기
def get_freq_word(tf_idf: dict, k: int):
    sort_docs_tf_idf = list(sorted(tf_idf.items(), key=lambda x: x[1], reverse=True))[:k]
    vocab = list(zip(*sort_docs_tf_idf))[0]
    vocab = list(vocab)
    vocab.sort()
    
    return vocab


# vocabulary 기반 voca-idx 딕셔너리 생성
def word_to_idx(words):
    voca_dict = {}
    for idx, voca in enumerate(words):
        voca_dict[voca] = idx
    
    return voca_dict


# 문장을 voca count로 임베딩(tf-idf 상위 k개의 vocabulary 기반 Count Vectorizer임베딩)
def embed_docs(docs: List, vocab: set):
    word2idx = word_to_idx(vocab)
    vec_dim = len(vocab)
    N = len(docs) # 문서 개수
    
    docs_embed = np.zeros((N, vec_dim)) # 문서임베딩 벡터의 모임 -> 행렬
    for i, doc in enumerate(docs):
        words = doc.split()
        for word in words:
            if word in vocab: # 단어가 vocabulary에 있을 때만
                idx = word2idx[word]
                docs_embed[i][idx] += 1
    
    return docs_embed

### b) 파일 읽는 함수 정의

In [3]:
import os
from tqdm import tqdm

# (X \t y)로 구성된 dataset을 읽어서 X, y로 각각 저장한다.
def load_data(data_dir, file_path):
    file = open(os.path.join(data_dir, file_path), encoding='UTF-8') #윈도우에서 cp949 파일 열기위해 encoding지정
    texts, labels = [], []

    for line in tqdm(file.readlines()):
        text, label = line.strip().split('\t')
        texts.append(text)
        labels.append(label)
    return texts, labels

# 2. 문서 Embedding

### a) TF-IDF vocab 구성
- 긍정 문서 상위 100개, 부정 문서 상위 100개 words 사용
- set(positive words + negative words)으로 구성된 vocab

In [4]:
train_X, train_y = load_data(os.path.join(os.getcwd(), 'data'), 'sentiment_train.txt')
test_X, test_y = load_data(os.path.join(os.getcwd(), 'data'), 'sentiment_test.txt')

100%|████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 300954.60it/s]


In [5]:
# Get positive/negative document respectively
pos_idx = [ i for i, y in enumerate(train_y) if y=="<P>"]
neg_idx = [ i for i, y in enumerate(train_y) if y=="<N>"]
train_pos_df = np.array(train_X)[pos_idx]
train_neg_df = np.array(train_X)[neg_idx]

train_pos_X, train_neg_X = "", ""
for pos, neg in zip(train_pos_df, train_neg_df):
    train_pos_X += " " + pos
    train_neg_X += " " + neg

# Get tf-idf of positive, negative document
tf_idf = get_TFIDF([train_pos_X, train_neg_X])

# Get top 100 frequent words respec.
top_words = 100
pos_words = get_freq_word(tf_idf[0] ,top_words)
neg_words = get_freq_word(tf_idf[1] ,top_words)

# Get total vocab (remove redundant words)
vocab = set(pos_words + neg_words)

In [6]:
print("전체 voca 개수: {} 개".format(len(vocab)))

전체 voca 개수: 200 개


### b) 문서 Embedding
- 추출한 TF-IDF 기반 문서 임베딩
- CountVectorizer 방식
- vocabulary에 없는 단어 제거, 빈 문자열 제거

In [7]:
# vocabulary에 없는 문자 제거
def postprocessing(docs: List, vocab: set) -> List:
    ret = []
    for doc in docs:
        string = ""
        for word in doc.split():
            if word in vocab: string += " " + word
        string = string.strip()
        ret.append(string)
    return ret

# 빈 문자열 제거
def remove_null(docs_X: List, docs_y: List):
    ret_X = []
    ret_y = []
    for idx in range(len(docs_X)):
        if docs_X[idx]: # 빈 문자열이 아닌 경우만 사용
            ret_X.append(docs_X[idx])
            ret_y.append(docs_y[idx])
    return ret_X, ret_y

In [8]:
train_X = postprocessing(train_X, vocab)
test_X = postprocessing(test_X, vocab)

train_X, train_y = remove_null(train_X, train_y)
test_X, test_y = remove_null(test_X, test_y)

In [9]:
print(len(train_X))
print(len(test_X))

744
104


In [10]:
# vocab을 기반으로 train_X를 임베딩한다.
train_X_embedding = embed_docs(train_X, vocab)

# 3. Naive Bayes 기반 문장 감성분류

### a) training

In [11]:
from sklearn.naive_bayes import MultinomialNB # Naive Bayes 모델

In [12]:
nb_clf = MultinomialNB()
nb_clf.fit(train_X_embedding, train_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### b) prediction on testset

In [13]:
# test는 transform만! (train voca로 transform 해야함)
test_X_embedding = embed_docs(test_X, vocab)
predictions = nb_clf.predict(test_X_embedding).tolist()

### c) 성능평가
- accuracy score

In [14]:
from sklearn.metrics import accuracy_score

print('Accuracy: %.5f' % accuracy_score(test_y, predictions))

Accuracy: 0.86538
